In [9]:
def calculate_intersect_union(pred, target):
    # calculate intersection and union
    intersection = np.logical_and(target, pred)
    union = np.logical_or(target, pred)

    return intersection, union

def calculate_iou(intersection, union):
    # calculate intersection over union
    return np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0

def calculate_accuracy(pred, target):
    # calculate accuracy
    return np.sum(pred == target) / np.prod(target.shape)

def calculate_dice(intersection, union):
    # calculate dice coefficient
    return 2 * np.sum(intersection) / (np.sum(union) + np.sum(intersection)) if np.sum(union) != 0 else 0

def iou(pred, target):
    # calculate intersection and union
    intersection, union = calculate_intersect_union(pred, target)
    # calculate intersection over union
    return calculate_iou(intersection, union)

def dice(pred, target):
    # calculate intersection and union
    intersection, union = calculate_intersect_union(pred, target)
    # calculate dice coefficient
    return calculate_dice(intersection, union)

def accuracy(pred, target):
    # calculate accuracy
    return calculate_accuracy(pred, target)

def compute_all(pred, target):

    # Calculate metrics using numpy arrays
    iou_value = iou(pred, target)
    dice_value = dice(pred, target)
    accuracy_value = accuracy(pred, target)

    metrics = {
        "IoU": iou_value,
        "Dice": dice_value,
        "Accuracy": accuracy_value,
    }

    return metrics

def compute_all(pred, target):

    # Calculate metrics using numpy arrays
    iou_value = iou(pred, target)
    dice_value = dice(pred, target)
    accuracy_value = accuracy(pred, target)

    metrics = {
        "IoU": iou_value,
        "Dice": dice_value,
        "Accuracy": accuracy_value,
    }

    return metrics

def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        
        
         # turn into Numpy arrays
        pred_label = np.array(pred_labels)
        label = np.array(labels)
        
        return compute_all(pred_label, label)

# TRANSFORMER

In [2]:
def get_files_filtered(path, name = "img.jpg"):
    image_files = []
    for file in find_files_by_pattern(path, name):
        # get npy file
        npy_path = os.path.join(os.path.dirname(file), "label.npy")
        array = np.load(npy_path)
        if np.sum(array) > 0:
            image_files.append(file)
    return image_files

def get_files_all(path, name = "img.jpg"):
    image_files = []
    for file in find_files_by_pattern(path, name):
        
        image_files.append(file)
        
    return image_files

In [52]:
transformer_model=SegformerForSemanticSegmentation.from_pretrained("/kaggle/input/tds-models/TDS_Models/best_model_transformer")

In [11]:
from datasets import Dataset, DatasetDict, Image

training_args = TrainingArguments(
    output_dir="car_segmentation",
    learning_rate=6e-5,
    num_train_epochs=20,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model='eval_Dice'
)

# Evaluating transformer 
folder_path = '/kaggle/input/car-street-data/train_test_split_car'

train_img_files = list(get_files_filtered(folder_path + "/train", 'img.jpg'))
train_label_files = list(get_files_filtered(folder_path + "/train", 'label.png'))

test_img_files = list(get_files_filtered(folder_path + "/test", 'img.jpg'))
test_label_files = list(get_files_filtered(folder_path + "/test", 'label.png'))

valid_img_files = list(get_files_filtered(folder_path + "/validation", 'img.jpg'))
valid_label_files = list(get_files_filtered(folder_path + "/validation", 'label.png'))
def create_dataset(image_paths, label_paths):
    dataset = Dataset.from_dict({"image": sorted(image_paths),
                                "label": sorted(label_paths)})
    dataset = dataset.cast_column("image", Image())
    dataset = dataset.cast_column("label", Image())

    return dataset
train_dataset = create_dataset(train_img_files, train_label_files)
test_dataset = create_dataset(test_img_files, test_label_files)
valid_dataset = create_dataset(valid_img_files, valid_label_files)
dataset = DatasetDict({'train': train_dataset, 'test': test_dataset, 'valid': valid_dataset})


val_transforms = A.Compose([
    A.Resize(height=500, width=500),
])

def preprocess_val_test(examples):
    augmented_pixel_values = []
    augmented_labels = []
    for i in range(len(examples["image"])):
        augmented = val_transforms(image=np.array(examples["image"][i]), mask=np.array(examples["label"][i]))
        
        # Convert to PyTorch tensors and floats
        pixel_values = torch.tensor(augmented["image"], dtype=torch.float32).permute(2, 0, 1) / 255.0
        labels = torch.tensor(augmented["mask"], dtype=torch.long)
        
        augmented_pixel_values.append(pixel_values)
        augmented_labels.append(labels)

    return {"pixel_values": augmented_pixel_values, "labels": augmented_labels}

dataset["valid"].set_transform(preprocess_val_test)

In [12]:
trainer = Trainer(
    model=transformer_car.model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["valid"] ,
    compute_metrics=compute_metrics,
)

trainer.evaluate(dataset["valid"])

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 0.005484578665345907,
 'eval_IoU': 0.6594682992284469,
 'eval_Dice': 0.7947946936197094,
 'eval_Accuracy': 0.99796975,
 'eval_runtime': 4.5824,
 'eval_samples_per_second': 10.475,
 'eval_steps_per_second': 2.619}

# FAST AI CAR

In [53]:
# Function to count .npy files with sum > 0
def count_files_with_sum_gt_zero(folder):
    count = 0
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.npy'):
                file_path = os.path.join(root, file)
                array = np.load(file_path)
                if np.sum(array) > 0:
                    count += 1
    return count


def find_files_by_pattern(folder, pattern):
    matching_files = []
    
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(pattern):
                matching_files.append(root + "/" + file)

    return matching_files

def get_images_filtered(name):
    image_files = []
    for file in find_files_by_pattern(folder_path, "img.jpg"):
        # get npy file
        npy_path = os.path.join(os.path.dirname(file), "label.npy")
        array = np.load(npy_path)
        if np.sum(array) > 0:
            image_files.append(Path(file))
    return image_files

def get_images_all(name):
    image_files = []
    for file in find_files_by_pattern(folder_path, "img.jpg"):
        image_files.append(Path(file))
    return image_files

# define a function to get the numpy mask for the given path
def get_mask(path):
    #remove the file name and extension from the path
    path = path.parent 
    # add the name mask.npy to the path
    path = path.joinpath("label.npy")
    return np.load(path)

aerial_Block_all = DataBlock(blocks = (ImageBlock, MaskBlock(codes = ["nothing","car"])),
                            splitter=GrandparentSplitter(train_name='train', valid_name='validation'),
                            get_items=get_images_all,
                            get_y=get_mask,
                            batch_tfms=aug_transforms(size=500, max_lighting=0.3))

dataloader_all = aerial_Block_all.dataloaders("./",bs=8)

# define functions to calculate desired metrics
def convert_preds_and_target(pred, target):
    # convert predictions to numpy array with max probability index
    pred = pred.argmax(dim=1).cpu().numpy()
    # convert target to numpy array
    target = target.cpu().numpy()

    return pred, target

def calculate_intersect_union(pred, target):
    # calculate intersection and union
    intersection = np.logical_and(target, pred)
    union = np.logical_or(target, pred)

    return intersection, union

def calculate_iou(intersection, union):
    # calculate intersection over union
    return np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 1

def calculate_accuracy(pred, target):
    # calculate accuracy
    return np.sum(pred == target) / np.prod(target.shape)

def calculate_dice(intersection, union):
    # calculate dice coefficient
    return 2 * np.sum(intersection) / (np.sum(union) + np.sum(intersection)) if np.sum(union) != 0 else 1

def iou(pred, target):
    # convert predictions and target to numpy arrays
    pred, target = convert_preds_and_target(pred, target)
    # calculate intersection and union
    intersection, union = calculate_intersect_union(pred, target)
    # calculate intersection over union
    return calculate_iou(intersection, union)

def dice(pred, target):
    # convert predictions and target to numpy arrays
    pred, target = convert_preds_and_target(pred, target)
    # calculate intersection and union
    intersection, union = calculate_intersect_union(pred, target)
    # calculate dice coefficient
    return calculate_dice(intersection, union)

def accuracy(pred, target):
    # convert predictions and target to numpy arrays
    pred, target = convert_preds_and_target(pred, target)
    # calculate accuracy
    return calculate_accuracy(pred, target)

learner_all = unet_learner(dataloader_all,resnet34,metrics=[dice, iou, accuracy])
learner_all.load('/kaggle/input/tds-models/TDS_Models/best_model_car')

In [55]:
results = learner_all.validate(1)
print(f"Results: {results}")

Results: [0.0013204907299950719, 0.7299018651494741, 0.617853217955704, 0.9995285263157893]


In [56]:
# Function to count .npy files with sum > 0
def count_files_with_sum_gt_zero(folder):
    count = 0
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.npy'):
                file_path = os.path.join(root, file)
                array = np.load(file_path)
                if np.sum(array) > 0:
                    count += 1
    return count


def find_files_by_pattern(folder, pattern):
    matching_files = []
    
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(pattern):
                matching_files.append(root + "/" + file)

    return matching_files

def get_images_filtered(name):
    image_files = []
    for file in find_files_by_pattern(folder_path, "img.jpg"):
        # get npy file
        npy_path = os.path.join(os.path.dirname(file), "label.npy")
        array = np.load(npy_path)
        if np.sum(array) > 0:
            image_files.append(Path(file))
    return image_files

def get_images_all(name):
    image_files = []
    for file in find_files_by_pattern(folder_path, "img.jpg"):
        image_files.append(Path(file))
    return image_files

# define a function to get the numpy mask for the given path
def get_mask(path):
    #remove the file name and extension from the path
    path = path.parent 
    # add the name mask.npy to the path
    path = path.joinpath("label_street_mask.npy")
    return np.load(path)

aerial_Block_all = DataBlock(blocks = (ImageBlock, MaskBlock(codes = ["nothing","car"])),
                            splitter=GrandparentSplitter(train_name='train', valid_name='validation'),
                            get_items=get_images_all,
                            get_y=get_mask,
                            batch_tfms=aug_transforms(size=500, max_lighting=0.3))

dataloader_all = aerial_Block_all.dataloaders("./",bs=8)

# define functions to calculate desired metrics
def convert_preds_and_target(pred, target):
    # convert predictions to numpy array with max probability index
    pred = pred.argmax(dim=1).cpu().numpy()
    # convert target to numpy array
    target = target.cpu().numpy()

    return pred, target

def calculate_intersect_union(pred, target):
    # calculate intersection and union
    intersection = np.logical_and(target, pred)
    union = np.logical_or(target, pred)

    return intersection, union

def calculate_iou(intersection, union):
    # calculate intersection over union
    return np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 1

def calculate_accuracy(pred, target):
    # calculate accuracy
    return np.sum(pred == target) / np.prod(target.shape)

def calculate_dice(intersection, union):
    # calculate dice coefficient
    return 2 * np.sum(intersection) / (np.sum(union) + np.sum(intersection)) if np.sum(union) != 0 else 1

def iou(pred, target):
    # convert predictions and target to numpy arrays
    pred, target = convert_preds_and_target(pred, target)
    # calculate intersection and union
    intersection, union = calculate_intersect_union(pred, target)
    # calculate intersection over union
    return calculate_iou(intersection, union)

def dice(pred, target):
    # convert predictions and target to numpy arrays
    pred, target = convert_preds_and_target(pred, target)
    # calculate intersection and union
    intersection, union = calculate_intersect_union(pred, target)
    # calculate dice coefficient
    return calculate_dice(intersection, union)

def accuracy(pred, target):
    # convert predictions and target to numpy arrays
    pred, target = convert_preds_and_target(pred, target)
    # calculate accuracy
    return calculate_accuracy(pred, target)

learner_all_street = unet_learner(dataloader_all,resnet34,metrics=[dice, iou, accuracy])
learner_all_street.load('/kaggle/input/tds-models/TDS_Models/best_model_street')
learner_all_street.validate(1)

/opt/conda/lib/python3.10/site-packages/fastai/learner.py:59: UserWarning: Saved file doesn't contain an optimizer state.
  elif with_opt: warn("Saved file doesn't contain an optimizer state.")


(#4) [0.04600682109594345,0.7963414279794532,0.6859894095144277,0.9834106947368421]